In [1]:
import tensorflow as tf
import numpy as np

# Load the data: IMDB movie review sentiment classification

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  23.7M      0  0:00:03  0:00:03 --:--:-- 23.7M


In [3]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [4]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [5]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [6]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [7]:
!rm -r aclImdb/train/unsup

In [8]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=2020,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=2020,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [9]:
print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")


Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [10]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b"This is one of the best movies I've ever seen. It has very good acting by Hanks, Newman, and everyone else. Definitely Jude Law's best performance. The cinematography is excellent, the editing is about as good, and includes a great original score that really fits in with the mood of the movie. The production design is also a factor in what makes this movie special. To me, it takes a lot to beat Godfather, but the fantastic cinematography displayed wins this contest. Definitely a Best Picture nominee in my book."
1
b"New York, I Love You, or rather should-be-titled Manhattan, I Love Looking At Your People In Sometimes Love, is a precise example of the difference between telling a story and telling a situation. Case in point, look at two of the segments in the film, one where Ethan Hawke lights a cigarette for a woman on a street and proceeds to chat her up with obnoxious sexy-talk, and another with Orlando Bloom trying to score a movie with an incredulous demand from his director to r

# Prepare the data

In [11]:
# remove <br /> tags
from tensorflow.keras.layers import TextVectorization
import string
import re

pattern = "<[^>]+>"
tf.strings.regex_replace("Text with tags.<br /><b>contains html</b>", pattern, " ")

<tf.Tensor: shape=(), dtype=string, numpy=b'Text with tags.  contains html '>

In [12]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, pattern, " ")
    return tf.strings.regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]", "")

In [13]:
# Model constants

max_features = 20000
embedding_dim = 128
sequence_length = 500

In [14]:
vectorize_layer = TextVectorization(
    max_tokens = max_features,
    standardize = custom_standardization,
    output_mode = "int",
    output_sequence_length = sequence_length
)

# vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

text_ds = raw_train_ds.map(lambda x, y: x) # no labels
vectorize_layer.adapt(text_ds)

In [29]:
# batch_size = 32
for train_dataset in raw_train_ds.take(1):
    print(train_dataset[0].numpy())

[b'The Bourne Ultimatum - Jason Bourne (Matt Damon in his best role ever), the newest spy kid on the block, brings his quest for his identity to a close as he also seeks to end the CIA\'s latest program "Blackbriar" to make super assassins like himself.<br /><br />I was so psyched for this one that I watched it\'s predecessors yesterday and today. Identity was as brilliant as I recall and Supremacy remains the weak (but still enjoyable) link in the chain for the weakest plot and, aside from a car chase which this film\'s chase easily tops, slight lacking in action and suspense.<br /><br />Hoo boy, does Ultimatum have suspense! Even when you know Bourne will escape the authorities (and boy do these films spotlight the police as inept), it\'s still brilliant watching him do it. It\'s mind-boggling to think that two guys with handguns and mopeds can create 10x more suspense than anything those $150 million giant robots did in Transformers.<br /><br />Chalk it up to Paul Greengrass, who ha

In [15]:
# Vectorize the data.
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [16]:
# Build a model

from tensorflow.keras import layers

inputs = layers.Input(shape=(None,), dtype="int64")

# Embedding
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

model.summary()

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 128)         114816    
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         114816    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                             

In [17]:
epochs = 3

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/3
625/625 [==============================] - 22s 21ms/step - loss: 0.5412 - accuracy: 0.6780 - val_loss: 0.3210 - val_accuracy: 0.8622
Epoch 2/3
625/625 [==============================] - 10s 16ms/step - loss: 0.2450 - accuracy: 0.9032 - val_loss: 0.3261 - val_accuracy: 0.8716
Epoch 3/3
625/625 [==============================] - 10s 16ms/step - loss: 0.1265 - accuracy: 0.9538 - val_loss: 0.3876 - val_accuracy: 0.8770


In [18]:
model.evaluate(test_ds)

782/782 [==============================] - 10s 12ms/step - loss: 0.4102 - accuracy: 0.8614


[0.41017910838127136, 0.8614400029182434]

In [19]:
inputs = tf.keras.Input(shape=(1,), dtype="string")
indices = vectorize_layer(inputs)
outputs = model(indices)


end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 11s 14ms/step - loss: 0.4102 - accuracy: 0.8614


[0.4101795256137848, 0.8614400029182434]